#### Code for train indexing
```bash
export i=0;
for img_path in $(find /home/alex/Downloads/train_dataset_mincult-train/train -name '*.jpg');
do \
    echo $i;
    i=$((i+1));
    echo $img_path;
    (echo -n '{"data":[{"blob":"'; base64 $img_path; echo '"}], "parameters": {"drop_image_content": true}, "execEndpoint":"/index"}') | curl -#H "Content-Type: application/json" -d @- http://192.168.125.50:51008/post | \
    jq -c '.data[] | .embedding' > $img_path.txt
done
```

#### Code for test searching
```bash
export i=0;
for img_path in $(find /home/alex/Downloads/test_dataset_mincult-test/test -name '*.jpg');
do \
    echo $i;
    i=$((i+1));
    echo $img_path;
    (echo -n '{"data":[{"blob":"'; base64 $img_path; echo '"}], "parameters": {"drop_image_content": true}, "execEndpoint":"/search"}') | curl -#H "Content-Type: application/json" -d @- http://192.168.125.50:51008/post > $img_path.txt
done
```

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import glob
import os
import json

# Create dataframe from train.csv + embeddings

In [2]:
train_root = '/home/alex/Downloads/train_dataset_mincult-train'

In [3]:
df = pd.read_csv(open(os.path.join(train_root, 'train.csv')), delimiter=';')
df

,object_id,name,description,group,img_name
0,10669820,Водолей - коник (фрагмент - голова),"сероглиняный, лепной, со сплошным белым ангобо...",Археология,7862029.jpg
1,4489444,Винтовка «Самозарядная винтовка Токарева» (мет...,"На стволе имеется надульник, на торце которог...",Оружие,9461061.jpg
2,8722586,Инструмент. Калибр-скоба,Прямоугольная пластина с усечёнными углами и д...,Прочие,5095122.jpg
3,3712248,"Судомодель. НИС ""Космонавт Виктор Пацаев"".","Корпус модели, надстройки, шлюпки выполнены и...",Прочие,551422.jpg
4,6339754,Сабля.,NaN,Оружие,2592073.jpg
...,...,...,...,...,...
20340,19945381,La Sainte Bible avec des explications & reflex...,"В переплете коричневой кожи, на корешке три н...",Редкие книги,20653985.jpg
20341,10334174,Фрагмент ручки синопской амфоры с сохранившимс...,NaN,Археология,41217662.jpg
20342,4708600,Шашка кавказская.(ножны),NaN,Оружие,1766049.jpg
20343,9622688,Висмутин,Висмутин. Зёрна в кварце,Минералогия,6305416.jpg


In [4]:
df['embedding'] = df.apply(lambda x: tuple(json.load(open(os.path.join(train_root, 
                                                                'train', 
                                                                str(x['object_id']), 
                                                                str(x['img_name'])) + '.txt', 
                                                   'r'))), 
                           axis=1).apply(np.array)
df

,object_id,name,description,group,img_name,embedding
0,10669820,Водолей - коник (фрагмент - голова),"сероглиняный, лепной, со сплошным белым ангобо...",Археология,7862029.jpg,"[0.658203125, 0.434814453125, -0.352783203125,..."
1,4489444,Винтовка «Самозарядная винтовка Токарева» (мет...,"На стволе имеется надульник, на торце которог...",Оружие,9461061.jpg,"[0.4580078125, 0.06298828125, -0.0270690917968..."
2,8722586,Инструмент. Калибр-скоба,Прямоугольная пластина с усечёнными углами и д...,Прочие,5095122.jpg,"[0.37353515625, -0.5947265625, 0.271484375, 0...."
3,3712248,"Судомодель. НИС ""Космонавт Виктор Пацаев"".","Корпус модели, надстройки, шлюпки выполнены и...",Прочие,551422.jpg,"[0.76025390625, -0.6123046875, 0.46875, 0.4721..."
4,6339754,Сабля.,NaN,Оружие,2592073.jpg,"[0.1309814453125, 0.76806640625, 0.08355712890..."
...,...,...,...,...,...,...
20340,19945381,La Sainte Bible avec des explications & reflex...,"В переплете коричневой кожи, на корешке три н...",Редкие книги,20653985.jpg,"[0.91259765625, 0.01474761962890625, -1.055664..."
20341,10334174,Фрагмент ручки синопской амфоры с сохранившимс...,NaN,Археология,41217662.jpg,"[1.0986328125, 0.305908203125, -0.193969726562..."
20342,4708600,Шашка кавказская.(ножны),NaN,Оружие,1766049.jpg,"[-0.0207672119140625, 0.3740234375, 0.58447265..."
20343,9622688,Висмутин,Висмутин. Зёрна в кварце,Минералогия,6305416.jpg,"[0.353515625, -0.732421875, -0.9111328125, -0...."


In [5]:
df['embedding']

0        [0.658203125, 0.434814453125, -0.352783203125,...
1        [0.4580078125, 0.06298828125, -0.0270690917968...
2        [0.37353515625, -0.5947265625, 0.271484375, 0....
3        [0.76025390625, -0.6123046875, 0.46875, 0.4721...
4        [0.1309814453125, 0.76806640625, 0.08355712890...
                               ...                        
20340    [0.91259765625, 0.01474761962890625, -1.055664...
20341    [1.0986328125, 0.305908203125, -0.193969726562...
20342    [-0.0207672119140625, 0.3740234375, 0.58447265...
20343    [0.353515625, -0.732421875, -0.9111328125, -0....
20344    [0.30712890625, -0.0631103515625, -0.641113281...
Name: embedding, Length: 20345, dtype: object

# Read test search results

In [6]:
test_root = '/home/alex/Downloads/test_dataset_mincult-test'

In [7]:
df_test = pd.read_csv(open(os.path.join(test_root, 'subm_test.csv')), delimiter=';')
df_test

,object_id,group,img_name
0,10239531,Техника,7273147.jpg
1,10239532,Техника,7273149.jpg
2,10314634,Техника,41197649.jpg
3,10314765,Техника,41218675.jpg
4,10332443,Техника,41222250.jpg
...,...,...,...
9984,48002959,Техника,59485929.jpg
9985,48509227,Техника,60173757.jpg
9986,48509237,Техника,60173762.jpg
9987,49040869,Техника,61029938.jpg


In [8]:
embs = np.vstack(df['embedding']).T

In [14]:
def find_group(x):
    try:
        emb = np.array(json.load(open(os.path.join(test_root, 'test', str(x['object_id']), str(x['img_name'])) + '.txt', 'r'))['data'][0]['matches'][0]['embedding'])
        emb = np.reshape(emb, (1, -1))
        similarity = emb @ embs
        idx = np.argmax(similarity)
        group = df['group'].iloc[idx]
        
        # group = df[df['embedding'] == str(json.load(open(os.path.join(test_root, 
                                                        #               'test', 
                                                        #               str(x['object_id']), 
                                                        #               str(x['img_name'])) + '.txt', 
                                                        #  'r'))['data'][0]['matches'][0]['embedding'])]['group'].iloc[0]
        return group
    except:
        print(str(x))
        return None

In [15]:
df_test['group'] = df_test.apply(lambda x: find_group(x), 
                                 axis=1)
df_test

,object_id,group,img_name
0,10239531,Техника,7273147.jpg
1,10239532,Археология,7273149.jpg
2,10314634,Археология,41197649.jpg
3,10314765,Археология,41218675.jpg
4,10332443,Археология,41222250.jpg
...,...,...,...
9984,48002959,"Фото, негативы",59485929.jpg
9985,48509227,"Фото, негативы",60173757.jpg
9986,48509237,"Фото, негативы",60173762.jpg
9987,49040869,"Фото, негативы",61029938.jpg


In [16]:
df_test.to_csv(os.path.join(test_root, 'test_dober2.csv'), sep = ';', encoding = 'utf-8', index = False)

In [17]:
df_test.group.value_counts()

group
Редкие книги                 1075
Нумизматика                   739
Естественнонауч.коллекция     730
Минералогия                   725
Печатная продукция            695
Техника                       688
Прочие                        682
ДПИ                           635
Документы                     623
Оружие                        621
Фото, негативы                615
Скульптура                    584
Археология                    553
Живопись                      516
Графика                       508
Name: count, dtype: int64

# Experiment code

In [10]:
emb = np.array(json.load(open(os.path.join(test_root, 'test', str(10333095), str('41198804.jpg')) + '.txt', 'r'))['data'][0]['matches'][0]['embedding'])
emb = np.reshape(emb, (1, -1))
emb

array([[ 1.20507812e+00,  2.10693359e-01, -8.07495117e-02,
        -2.09838867e-01, -3.52294922e-01, -5.49804688e-01,
         8.22753906e-02, -4.06250000e-01,  3.25439453e-01,
         6.92749023e-02,  3.13720703e-01,  3.74267578e-01,
         5.13183594e-01,  2.21069336e-01,  1.84082031e-01,
        -1.08108521e-02, -8.24707031e-01, -4.04296875e-01,
         5.15625000e-01, -1.77734375e-01,  2.55859375e-01,
        -2.18383789e-01,  5.42968750e-01,  3.92578125e-01,
        -7.60498047e-02,  1.99584961e-01, -8.48388672e-02,
         4.12597656e-01,  6.65283203e-03,  8.31909180e-02,
         8.63037109e-02, -1.81762695e-01, -1.13769531e+00,
        -1.11816406e+00,  3.03222656e-01,  9.98535156e-02,
         2.11303711e-01,  7.26318359e-02, -3.57421875e-01,
         4.19189453e-01, -4.61425781e-01,  2.55615234e-01,
         2.47436523e-01, -3.18603516e-01,  1.39648438e-01,
        -6.47583008e-02, -2.35839844e-01, -3.69140625e-01,
        -2.56591797e-01, -1.55151367e-01,  2.76367188e+0

In [13]:
similarity = emb @ embs
idx = np.argmax(similarity)
print(similarity.shape)
print(idx)
print(similarity[0, idx])
print(df['group'].iloc[idx])
df.iloc[idx]

(1, 20345)
12398
190.68762568472175
Археология


object_id                                               10332807
name                  Край горла гераклейской амфоры с клеймом. 
description                                                  NaN
group                                                 Археология
img_name                                            41226883.jpg
embedding      [1.205078125, 0.210693359375, -0.0807495117187...
Name: 12398, dtype: object